In [29]:
import numpy as np 
import math 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.integrate as integrate
from scipy.optimize import fsolve

In [2]:
data_1 = {
    'Maturity': [0, 1, 3, 5, 7, 10],
    'CDS_rate': [np.nan, 100, 110, 120, 120, 125]
}
cds_rate = pd.DataFrame(data_1)
cds_rate

,Maturity,CDS_rate
0,0,NaN
1,1,100.0
2,3,110.0
3,5,120.0
4,7,120.0
5,10,125.0


# Question 1

In [3]:
def average_hazard_rate(cds_rate):
    return cds_rate / LGD / 10000

In [4]:
def survival_prob(maturity, average_hazard_rate):
    return np.exp(-average_hazard_rate * maturity)

In [5]:
LGD = 0.4
h_rate = []

for i in range(cds_rate.shape[0]):
    maturity = cds_rate['Maturity'].iloc[i] 
    cds_rate_value = cds_rate['CDS_rate'].iloc[i]  
    average_hazard_rate_value = average_hazard_rate(cds_rate_value)
    h_rate.append({'Maturity': maturity, 'Average lamda': average_hazard_rate_value})

h_rate_df = pd.DataFrame(h_rate)
h_rate_df

,Maturity,Average lamda
0,0,NaN
1,1,0.02500
2,3,0.02750
3,5,0.03000
4,7,0.03000
5,10,0.03125


In [6]:
h_rate_df['Q(t>T)'] = survival_prob(h_rate_df['Maturity'], h_rate_df['Average lamda'])
h_rate_df.iloc[0, 2] = 1

In [7]:
h_rate_df['fwd_h_rate'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'fwd_h_rate'] = -np.log(h_rate_df.loc[i, 'Q(t>T)'] / h_rate_df.loc[i - 1, 'Q(t>T)'])\
    / (h_rate_df.loc[i, 'Maturity'] - h_rate_df.loc[i - 1, 'Maturity'])       

In [8]:
h_rate_df['default_prob'] = 0
for i in range(1, h_rate_df.shape[0]):
    h_rate_df.loc[i, 'default_prob'] = h_rate_df.loc[i - 1, 'Q(t>T)'] - h_rate_df.loc[i, 'Q(t>T)']


In [142]:
h_rate_df

,Maturity,Average lamda,Q(t>T),fwd_h_rate,default_prob
0,0,NaN,1.000000,0.000000,0.000000
1,1,0.02500,0.975310,0.025000,0.024690
2,3,0.02750,0.920811,0.028750,0.054498
3,5,0.03000,0.860708,0.033750,0.060103
4,7,0.03000,0.810584,0.030000,0.050124
5,10,0.03125,0.731616,0.034167,0.078969


# Question 2

In [100]:
from scipy.optimize import fsolve

def getCumulLambdas(r, t, rec, dSpread, payment):
    """
    returns a list (or list of lists) of cumulative hazard rates
    
    r: interest rate
    t: tenor of CDS
    rec: recovery rate
    dSpread: spread of CDS (please enter it by dividing it by 10000 (bps))
    payment: 4 if quarterly, 1 if annually etc
    """
    
    vPaymentTimes = np.linspace(0, t, (t * payment) + 1)
    dInitialLambda = dSpread / (1 - rec) 
    
    def calcDiff(h, r, t, vPaymentTimes, rec, dSpread, payment):
        vPremiumPayments = np.ones(len(vPaymentTimes))
        for j, i in enumerate(vPaymentTimes):
            vPremiumPayments[j] = np.exp(-r * i) * (1 / payment) * np.exp(-h * i)
        dPremiumPayment = np.sum(vPremiumPayments)
        
        def calcIntegrandAccrued(u, r, h):
            dIntegrandAccrued = np.exp(-r * u) * ((1 / payment) / 2) * h * np.exp(-h * u)
            return dIntegrandAccrued
        
        vIaccrued = integrate.quad(calcIntegrandAccrued, 0, t, args=(r, h))
        dAccruedPayment = vIaccrued[0]
        
        def calcIntegrandProtection(u, r, h):
            dIntegrandProtection = np.exp(-r * u) * h * np.exp(-h * u)
            return dIntegrandProtection
        
        vIprotection = integrate.quad(calcIntegrandProtection, 0, t, args=(r, h))
        dProtectionPayment = vIprotection[0]
        
        dPremiumLeg = dSpread * (dPremiumPayment + dAccruedPayment)
        dProtectionLeg = (1 - rec) * dProtectionPayment 
        
        return dPremiumLeg - dProtectionLeg
    
    def root_function(h, *args):
        return calcDiff(h, *args)
    
    cumulative_lambda = fsolve(root_function, dInitialLambda, args=(r, t, vPaymentTimes, rec, dSpread, payment))
    
    return cumulative_lambda


In [101]:
r = 0.03
dSpread = 100 / 10000
t = 3
rec = 0.6
payment = 4
getCumulLambdas(r, t, rec, dSpread, payment)

array([0.02717378])

### Lambda 1

In [105]:
def get_lambda_1(r, rate, T_start, T_stop, LGD, payment):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment):
        # Define the payment times
        payment_times = np.linspace(T_start, T_stop, int((T_stop - T_start) * payment) + 1)
    
        # Premium if not default
        init_premium = np.zeros(len(payment_times))
        for i, u in enumerate(payment_times):
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r*u) * (1/payment) * np.exp(-h*u)
        final_premium = np.sum(init_premium)
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                init_acc_premium[i] = rate * np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * ((1/payment)/2) * \
                (np.exp(-h*payment_times[i-1]) - np.exp(-h*payment_times[i]))
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_protection[i] = 0
            else:
                init_protection[i] = np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * \
                (np.exp(-h*payment_times[i-1]) - np.exp(-h*payment_times[i]))
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate/LGD/10000, args=(r, rate, T_start, T_stop, LGD, payment))
    
    return cumulative_lambda


In [106]:
r = 0.03
rate = 100 / 10000
T_start = 0
T_stop = 1
LGD = 0.4
payment = 4
h_1 = get_lambda_1(r, rate, T_start, T_stop, LGD, payment)[0]
h_1

0.024906797363837867

### Lambda 2

In [107]:
def get_lambda_2(r, rate, T_start, T_stop, LGD, payment, h_1):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1):
        
        # Define the payment times
        payment_times = np.linspace(T_start, T_stop, int((T_stop - T_start) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if payment_times[i] <= 1:
                h_func = -h_1*payment_times[i]    
            else:
                h_func = -h*payment_times[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r*payment_times[i]) * (1/payment) * np.exp(h_func)      
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                if payment_times[i] <= 1:
                    h_func_1 = -h_1*payment_times[i-1]
                    h_func_2 = -h_1*payment_times[i]    
                else:
                    h_func_1 = -h*payment_times[i-1]
                    h_func_2 = -h*payment_times[i]   
                init_acc_premium[i] = rate * np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * ((1/payment)/2) * \
                (np.exp(h_func_1) - np.exp(h_func_2))     
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times))
        for i in range(len(payment_times)):
            if i == 0:
                init_protection[i] = 0
            else:
                if payment_times[i] <= 1:
                    h_func_1 = -h_1*payment_times[i-1]
                    h_func_2 = -h_1*payment_times[i]
                else:
                    h_func_1 = -h*payment_times[i-1]
                    h_func_2 = -h*payment_times[i]
                init_protection[i] = np.exp(-r*(payment_times[i] + payment_times[i-1])/2) * \
                (np.exp(h_func_1) - np.exp(h_func_2))
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1))
    
    return cumulative_lambda


In [108]:
r = 0.03
rate_3 = 110 / 10000
T_start = 0
T_stop = 3
LGD = 0.4
payment = 4
h_2 = get_lambda_2(r, rate_3, T_start, T_stop, LGD, payment, h_1)[0]

In [109]:
h_2

0.02875965289507548

### Lambda 3

In [113]:
def get_lambda_3(r, rate, T_start, T_stop, LGD, payment, h_1, h_2):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, T_stop, int((T_stop - 3) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            h_func = -h * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h * payment_times_3[i-1]
                h_func_2 = -h * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, T_stop]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h * payment_times_3[i-1]
                h_func_2 = -h * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2))
    
    return cumulative_lambda

In [116]:
r = 0.03
rate_3 = 120 / 10000
T_start = 0
T_stop = 5
LGD = 0.4
payment = 4
h_3 = get_lambda_3(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2)[0]

In [117]:
h_3

0.03431854434859217

### Lambda 4

In [119]:
def get_lambda_4(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, 5, int(2 * payment) + 1)
        payment_times_4 = np.linspace(5, T_stop, int((T_stop - 5) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            h_func = -h_3 * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            h_func = -h * payment_times_4[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * payment_times_4[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h * payment_times_4[i-1]
                h_func_2 = -h * payment_times_4[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [5, T_stop]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h * payment_times_4[i-1]
                h_func_2 = -h * payment_times_4[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3))
    
    return cumulative_lambda


In [120]:
r = 0.03
rate_3 = 120 / 10000
T_start = 0
T_stop = 7
LGD = 0.4
payment = 4
h_4 = get_lambda_4(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2, h_3)[0]

In [121]:
h_4

0.02988826924767897

### Lambda 5

In [123]:
def get_lambda_5(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4):
    
    def get_diff(h, r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4):
        
        # Define the payment times
        payment_times_1 = np.linspace(T_start, 1, int(1 * payment) + 1)
        payment_times_2 = np.linspace(1, 3, int(2 * payment) + 1)
        payment_times_3 = np.linspace(3, 5, int(2 * payment) + 1)
        payment_times_4 = np.linspace(5, 7, int(2 * payment) + 1)
        payment_times_5 = np.linspace(7, T_stop, int((T_stop - 7) * payment) + 1)
        
        # Premium if not default
        init_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            h_func = -h_1 * payment_times_1[i]
            if i == 0:
                init_premium[i] = 0
            else:
                init_premium[i] = rate * np.exp(-r * payment_times_1[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            h_func = -h_2 * payment_times_2[i]
            if i == 0:
                init_premium[i + len(payment_times_1)] = 0
            else:
                init_premium[i + len(payment_times_1)] = rate * np.exp(-r * payment_times_2[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            h_func = -h_3 * payment_times_3[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * payment_times_3[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            h_func = -h_4 * payment_times_4[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * payment_times_4[i]) * (1 / payment) * np.exp(h_func)
                
        # Premium leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            h_func = -h * payment_times_5[i]
            if i == 0:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                init_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = rate * np.exp(-r * payment_times_5[i]) * (1 / payment) * np.exp(h_func)
                
        final_premium = np.sum(init_premium)    
    
        # Accrued Premium if default
        init_acc_premium = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Accrued premium leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_acc_premium[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_acc_premium[i] = rate * np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))     
                
        # Accrued premium leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_acc_premium[i + len(payment_times_1)] = rate * np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))  
                
        # Accrued premium leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2)] = rate * np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h_4 * payment_times_4[i-1]
                h_func_2 = -h_4 * payment_times_4[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = rate * np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Accrued premium leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            if i == 0:
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                h_func_1 = -h * payment_times_5[i-1]
                h_func_2 = -h * payment_times_5[i]
                init_acc_premium[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = rate * np.exp(-r * (payment_times_5[i] + payment_times_5[i-1]) / 2) * ((1 / payment) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_acc_premium = np.sum(init_acc_premium)
    
        # Protection leg
        init_protection = np.zeros(len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4) + len(payment_times_5))
        
        # Protection leg for time range [0, 1]
        for i in range(len(payment_times_1)):
            if i == 0:
                init_protection[i] = 0
            else:
                h_func_1 = -h_1 * payment_times_1[i-1]
                h_func_2 = -h_1 * payment_times_1[i]
                init_protection[i] = np.exp(-r * (payment_times_1[i] + payment_times_1[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [1, 3]
        for i in range(len(payment_times_2)):
            if i == 0:
                init_protection[i + len(payment_times_1)] = 0
            else:
                h_func_1 = -h_2 * payment_times_2[i-1]
                h_func_2 = -h_2 * payment_times_2[i]
                init_protection[i + len(payment_times_1)] = np.exp(-r * (payment_times_2[i] + payment_times_2[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [3, 5]
        for i in range(len(payment_times_3)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = 0
            else:
                h_func_1 = -h_3 * payment_times_3[i-1]
                h_func_2 = -h_3 * payment_times_3[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2)] = np.exp(-r * (payment_times_3[i] + payment_times_3[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [5, 7]
        for i in range(len(payment_times_4)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = 0
            else:
                h_func_1 = -h_4 * payment_times_4[i-1]
                h_func_2 = -h_4 * payment_times_4[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3)] = np.exp(-r * (payment_times_4[i] + payment_times_4[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        # Protection leg for time range [7, T_stop]
        for i in range(len(payment_times_5)):
            if i == 0:
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = 0
            else:
                h_func_1 = -h * payment_times_5[i-1]
                h_func_2 = -h * payment_times_5[i]
                init_protection[i + len(payment_times_1) + len(payment_times_2) + len(payment_times_3) + len(payment_times_4)] = np.exp(-r * (payment_times_5[i] + payment_times_5[i-1]) / 2) * \
                    (np.exp(h_func_1) - np.exp(h_func_2))
                
        final_protection = np.sum(init_protection)
        
        protection_leg = LGD * final_protection
    
        # Difference
        diff = final_premium + final_acc_premium - protection_leg
        
        return diff

    # Use a solver to find the cumulative hazard rate that satisfies the equation
    cumulative_lambda = fsolve(get_diff, rate / LGD / 10000, args=(r, rate, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4))
    
    return cumulative_lambda


In [125]:
r = 0.03
rate_3 = 125 / 10000
T_start = 0
T_stop = 10
LGD = 0.4
payment = 4
h_5 = get_lambda_5(r, rate_3, T_start, T_stop, LGD, payment, h_1, h_2, h_3, h_4)[0]

In [126]:
h_5

0.035244860332574006